# 🔬 CNN Activation Analyzer

**Herramienta de visualización e interpretación de activaciones en redes convolucionales**

## ¿Qué hace este notebook?

Permite **analizar y visualizar las activaciones internas** de CNNs para entender:
- 🎯 Qué patrones detecta cada filtro (bordes, texturas, formas)
- 📊 Qué neuronas se activan ante imágenes específicas
- 🔍 Cómo la red procesa información capa por capa
- 🎨 Descomposición RGB de filtros convolucionales

## Modelos soportados

- **ResNet-18**: Red residual de 18 capas
- **AlexNet**: Red clásica pionera en ImageNet

## Uso rápido

1. Carga una imagen
2. Selecciona modelo y capa
3. Analiza las activaciones
4. Visualiza los filtros más relevantes

---

In [ ]:

# ===================================================================
# CELDA 1: Imports y Setup
# ===================================================================

import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import torch


sys.path.append('..')

from src.models.model_loader import ModelLoader
from src.utils.image_analyzer import SingleImageAnalyzer, analyze_single_image

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")

# Cargar modelo
print("\n📥 Cargando modelo...")
# loader = ModelLoader('resnet18', pretrained=True, device=device)
loader = ModelLoader('alexnet', pretrained=True, device=device)
model = loader.load_model()
print("✅ Modelo cargado")

In [ ]:
# ===================================================================
# CELDA 2: Análisis Rápido con Logs (VERSIÓN SIMPLIFICADA)
# ===================================================================

from pathlib import Path
import requests
from PIL import Image
from io import BytesIO

# Descargar imagen de ejemplo
data_dir = Path("../data/test_images")
data_dir.mkdir(parents=True, exist_ok=True)

image_url = "https://images.unsplash.com/photo-1574158622682-e40e69881006?w=400"
image_path = data_dir / "cat_example.jpg"

if not image_path.exists():
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    img.save(image_path)

# 🎯 CONFIGURACIÓN
target_layer = 'features.0'  # Cambia esto: 'conv1', 'layer1.0.conv1', 'layer2.0.conv1'

# ✅ ANÁLISIS CON LOGS AUTOMÁTICOS
from src.utils.image_analyzer import analyze_and_visualize_layer

results = analyze_and_visualize_layer(
    model=model,
    image_path=str(image_path),
    target_layer=target_layer,
    device=device,
    top_k=12,
    figsize=(20, 14),
    cmap='jet',
    show_image=True,
    verbose=True  # ← Activa logs detallados
)

In [ ]:
# ===================================================================
# CELDA 3: Análisis Detallado con Control Manual
# ===================================================================

# Para más control, usar la clase directamente
analyzer = SingleImageAnalyzer(model, target_layer=target_layer, device=device)

# Cargar imagen
img_tensor, img_vis = analyzer.load_image(image_path)

# Analizar
results = analyzer.analyze_image(img_tensor)
activations = results['activations']

print(f"\n📊 Shape de activaciones: {activations.shape}")
print(f"   [batch, canales, height, width]")

# Estadísticas por neurona
stats = analyzer.get_neuron_statistics(activations)

# Top 10 por activación promedio
top_by_mean = analyzer.get_top_neurons(stats, top_k=10, criterion='mean')
print(f"\n🏆 Top 10 neuronas por activación PROMEDIO:")
for rank, idx in enumerate(top_by_mean, 1):
    s = stats[idx]
    print(f"   {rank:2d}. Neurona {s['neuron_idx']:3d}: "
          f"mean={s['mean']:7.4f}, max={s['max']:7.4f}, "
          f"sparsity={s['sparsity']*100:5.1f}%")

# Top 10 por activación máxima
top_by_max = analyzer.get_top_neurons(stats, top_k=10, criterion='max')
print(f"\n🔥 Top 10 neuronas por activación MÁXIMA:")
for rank, idx in enumerate(top_by_max, 1):
    s = stats[idx]
    print(f"   {rank:2d}. Neurona {s['neuron_idx']:3d}: "
          f"max={s['max']:7.4f}, mean={s['mean']:7.4f}")

In [ ]:
# ===================================================================
# CELDA 4: Visualizar Neuronas Específicas (VERSIÓN DINÁMICA)
# ===================================================================

print("="*70)
print("🎨 VISUALIZACIÓN DE NEURONAS ESPECÍFICAS")
print("="*70)

# ===================================================================
# CONFIGURACIÓN: Elige tu estrategia
# ===================================================================

# Estrategia de selección
STRATEGY = 'top_mean'  # Opciones: 'top_mean', 'top_max', 'manual', 'mixed', 'interesting'

# Número de neuronas a visualizar
NUM_NEURONS = 12

# Solo si STRATEGY='manual'
MANUAL_NEURONS = [0, 5, 10, 15, 20, 25, 30, 35]

# Configuración visual
COLORMAP = 'jet'  # Opciones: 'jet', 'hot', 'viridis', 'plasma', 'magma', 'coolwarm'
FIGSIZE = (20, 12)

# ===================================================================
# SELECCIÓN AUTOMÁTICA SEGÚN ESTRATEGIA
# ===================================================================

if STRATEGY == 'top_mean':
    # Top N por activación promedio
    selected_neurons = top_by_mean[:NUM_NEURONS]
    title_suffix = f"Top {NUM_NEURONS} por Activación Promedio"
    print(f"\n✅ Estrategia: TOP MEAN")
    print(f"   Neuronas seleccionadas: {selected_neurons}")

elif STRATEGY == 'top_max':
    # Top N por activación máxima
    selected_neurons = top_by_max[:NUM_NEURONS]
    title_suffix = f"Top {NUM_NEURONS} por Activación Máxima"
    print(f"\n✅ Estrategia: TOP MAX")
    print(f"   Neuronas seleccionadas: {selected_neurons}")

elif STRATEGY == 'manual':
    # Selección manual
    selected_neurons = MANUAL_NEURONS[:NUM_NEURONS]
    title_suffix = "Neuronas Seleccionadas Manualmente"
    print(f"\n✅ Estrategia: MANUAL")
    print(f"   Neuronas seleccionadas: {selected_neurons}")

elif STRATEGY == 'mixed':
    # Mitad top_mean, mitad top_max (sin duplicados)
    half = NUM_NEURONS // 2
    top_mean_subset = top_by_mean[:half]
    top_max_subset = [n for n in top_by_max if n not in top_mean_subset][:NUM_NEURONS - half]
    selected_neurons = top_mean_subset + top_max_subset
    title_suffix = f"Mix: Top por Mean y Max"
    print(f"\n✅ Estrategia: MIXED")
    print(f"   Top por mean: {top_mean_subset}")
    print(f"   Top por max:  {top_max_subset}")

elif STRATEGY == 'interesting':
    # Neuronas "interesantes": alta activación pero también sparse
    interesting = []
    for s in stats:
        # Criterio: activación alta + cierta selectividad
        if s['mean'] > 0.3 and s['max'] > 2.0:
            interesting.append(s['neuron_idx'])
    
    selected_neurons = interesting[:NUM_NEURONS]
    title_suffix = "Neuronas 'Interesantes' (alta activación + selectividad)"
    print(f"\n✅ Estrategia: INTERESTING")
    print(f"   {len(interesting)} neuronas encontradas")
    print(f"   Mostrando: {selected_neurons}")

else:
    print(f"\n❌ Estrategia '{STRATEGY}' no reconocida")
    print(f"   Opciones: 'top_mean', 'top_max', 'manual', 'mixed', 'interesting'")
    selected_neurons = top_by_mean[:NUM_NEURONS]
    title_suffix = "Default (Top by Mean)"

# ===================================================================
# MOSTRAR INFO DE NEURONAS SELECCIONADAS
# ===================================================================

print(f"\n📊 ESTADÍSTICAS DE NEURONAS SELECCIONADAS:")
print(f"   {'Neurona':>8} | {'Media':>10} | {'Máxima':>10} | {'Std':>10} | {'Sparsity':>10}")
print(f"   {'-'*65}")

for neuron_idx in selected_neurons:
    s = stats[neuron_idx]
    print(f"   {neuron_idx:8d} | {s['mean']:10.4f} | {s['max']:10.4f} | "
          f"{s['std']:10.4f} | {s['sparsity']*100:9.1f}%")

# ===================================================================
# VISUALIZACIÓN
# ===================================================================

print(f"\n🎨 Generando visualización con colormap '{COLORMAP}'...")

# Modificar título del gráfico
fig = plt.figure(figsize=FIGSIZE)
gs = fig.add_gridspec(4, 4, hspace=0.35, wspace=0.3)

# Título principal
fig.suptitle(
    f'Mapas de Activación - {analyzer.actual_layer_name}\n{title_suffix}',
    fontsize=14,
    fontweight='bold',
    y=0.98
)

# Imagen original
ax_img = fig.add_subplot(gs[0, :])
ax_img.imshow(img_vis)
ax_img.set_title('Imagen Original', fontsize=13, fontweight='bold', pad=15)
ax_img.axis('off')

# Mapas de activación
for idx, neuron_idx in enumerate(selected_neurons):
    row = (idx // 4) + 1
    col = idx % 4
    
    ax = fig.add_subplot(gs[row, col])
    
    # Obtener mapa
    act_map = activations[0, neuron_idx, :, :].cpu().numpy()
    
    # Normalizar
    if act_map.max() > act_map.min():
        act_map_norm = (act_map - act_map.min()) / (act_map.max() - act_map.min())
    else:
        act_map_norm = act_map
    
    # Mostrar
    im = ax.imshow(act_map_norm, cmap=COLORMAP, interpolation='bilinear')
    
    # Título con estadísticas
    s = stats[neuron_idx]
    ax.set_title(
        f'Neurona {neuron_idx}\n'
        f'μ={s["mean"]:.3f}, max={s["max"]:.3f}',
        fontsize=9,
        fontweight='bold'
    )
    ax.axis('off')
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=7)

plt.show()

print(f"\n✅ Visualización completada")
print("="*70)

In [ ]:
# ===================================================================
# CELDA 5: Overlay de Activación sobre Imagen
# ===================================================================

# Crear overlays para top 6 neuronas
top_neurons = analyzer.get_top_neurons(stats, top_k=6)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, neuron_idx in enumerate(top_neurons):
    overlay = analyzer.get_activation_overlay(
        image_vis=img_vis,
        activations=activations,
        neuron_idx=neuron_idx,
        alpha=0.6
    )
    
    axes[idx].imshow(overlay)
    
    s = stats[neuron_idx]
    axes[idx].set_title(f'Neurona {neuron_idx}\n'
                       f'mean={s["mean"]:.3f}, max={s["max"]:.3f}',
                       fontsize=10, fontweight='bold')
    axes[idx].axis('off')

plt.suptitle(f'Overlays de Activación - {target_layer}', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ===================================================================
# CELDA 6: Comparar Múltiples Capas (CON TÍTULOS MEJORADOS)
# ===================================================================

# layers_to_compare = ['conv1', 'layer1.0.conv1', 'layer2.0.conv1', 'layer3.0.conv1']
layers_to_compare = ['features.0', 'features.3', 'features.6', 'features.8']

fig, axes = plt.subplots(
    len(layers_to_compare), 
    5,  # 5 columnas: 1 para título + 4 para neuronas
    figsize=(20, 4*len(layers_to_compare)),
    gridspec_kw={'width_ratios': [1, 4, 4, 4, 4]}  # Primera columna más estrecha
)

for layer_idx, layer_name in enumerate(layers_to_compare):
    print(f"\n🔍 Analizando {layer_name}...")
    
    # Crear analizador para esta capa
    analyzer_temp = SingleImageAnalyzer(model, layer_name, device)
    
    # Analizar
    results_temp = analyzer_temp.analyze_image(img_tensor)
    acts = results_temp['activations']
    
    print(f"   Shape de activaciones: {acts.shape}")
    
    # Estadísticas
    stats_temp = analyzer_temp.get_neuron_statistics(acts)
    top_4 = analyzer_temp.get_top_neurons(stats_temp, top_k=4)
    
    # ✅ COLUMNA 0: TÍTULO DE LA FILA (nombre de capa)
    axes[layer_idx, 0].text(
        0.5, 0.5, 
        f'{layer_name}\n\n'
        f'Shape:\n{acts.shape[1]} canales\n'
        f'{acts.shape[2]}×{acts.shape[3]} píxeles',
        ha='center', 
        va='center',
        fontsize=11,
        fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8)
    )
    axes[layer_idx, 0].axis('off')
    
    # ✅ COLUMNAS 1-4: MAPAS DE ACTIVACIÓN
    for col_idx, neuron_idx in enumerate(top_4):
        act_map = acts[0, neuron_idx, :, :].cpu().numpy()
        
        # Normalizar
        if act_map.max() > act_map.min():
            act_map_norm = (act_map - act_map.min()) / (act_map.max() - act_map.min())
        else:
            act_map_norm = act_map
        
        # Mostrar heatmap
        im = axes[layer_idx, col_idx + 1].imshow(act_map_norm, cmap='jet')
        
        # Título individual
        axes[layer_idx, col_idx + 1].set_title(
            f'Neurona {neuron_idx}\nμ={act_map.mean():.2f}',
            fontsize=9,
            fontweight='bold'
        )
        axes[layer_idx, col_idx + 1].axis('off')
        
        # Colorbar
        plt.colorbar(im, ax=axes[layer_idx, col_idx + 1], fraction=0.046, pad=0.04)
    
    analyzer_temp.cleanup()

plt.suptitle('Comparación de Activaciones entre Capas\n(Top 4 neuronas por capa)', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("✅ Comparación completada")
print("="*70)

In [ ]:
# ===================================================================
# CELDA: Visualizar FILTROS (Patrones que busca la neurona)
# ===================================================================

import torch
import numpy as np
import matplotlib.pyplot as plt

print("="*70)
print("🔍 VISUALIZACIÓN DE FILTROS CONVOLUCIONALES")
print("="*70)

# Seleccionar capa
# target_layer_name = 'layer2.0.conv1'  # Cambia esto

# Obtener el módulo de la capa
layer_module = dict(model.named_modules())[layers_to_compare[0]]

# Obtener los pesos (filtros)
filters = layer_module.weight.data.cpu()  # Shape: [out_channels, in_channels, kernel_h, kernel_w]

print(f"\n📊 Información de la capa '{target_layer_name}':")
print(f"   Shape de filtros: {filters.shape}")
print(f"   ├─ Num filtros (out):  {filters.shape[0]}")
print(f"   ├─ Canales de entrada: {filters.shape[1]}")
print(f"   └─ Tamaño kernel:      {filters.shape[2]}×{filters.shape[3]}")

# ===================================================================
# FUNCIÓN: Normalizar filtro para visualización
# ===================================================================

def normalize_filter(filter_tensor):
    """Normaliza un filtro para visualización en [0, 1]"""
    fmin = filter_tensor.min()
    fmax = filter_tensor.max()
    if fmax - fmin > 0:
        return (filter_tensor - fmin) / (fmax - fmin)
    return filter_tensor


# ===================================================================
# VISUALIZACIÓN: Primeros 16 filtros
# ===================================================================

num_filters_to_show = min(16, filters.shape[0])
num_input_channels = filters.shape[1]

print(f"\n🎨 Visualizando {num_filters_to_show} filtros...")

if num_input_channels == 3:
    # ===================================================================
    # CASO 1: Filtros RGB (solo conv1)
    # ===================================================================
    
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.flatten()
    
    for i in range(num_filters_to_show):
        filter_rgb = filters[i]  # [3, kernel_h, kernel_w]
        filter_vis = filter_rgb.permute(1, 2, 0).numpy()  # [kernel_h, kernel_w, 3]
        filter_vis = normalize_filter(torch.from_numpy(filter_vis)).numpy()
        
        axes[i].imshow(filter_vis, interpolation='nearest')
        axes[i].set_title(f'Filtro {i}', fontsize=9)
        axes[i].axis('off')
    
    plt.suptitle(f'Filtros RGB de {target_layer_name}\n(Patrones de 7×7 que busca cada neurona)',
                fontsize=13, fontweight='bold')

else:
    # ===================================================================
    # CASO 2: Filtros de capas profundas (muchos canales de entrada)
    # ===================================================================
    
    print(f"\n⚠️  Esta capa tiene {num_input_channels} canales de entrada")
    print(f"   No se pueden visualizar como RGB")
    print(f"   Mostrando promedio de canales de entrada...\n")
    
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.flatten()
    
    for i in range(num_filters_to_show):
        # Promediar sobre canales de entrada
        filter_avg = filters[i].mean(dim=0).numpy()  # [kernel_h, kernel_w]
        filter_vis = normalize_filter(torch.from_numpy(filter_avg)).numpy()
        
        axes[i].imshow(filter_vis, cmap='gray', interpolation='nearest')
        axes[i].set_title(f'Filtro {i}', fontsize=9)
        axes[i].axis('off')
    
    plt.suptitle(
        f'Filtros de {target_layer_name}\n'
        f'(Promedio de {num_input_channels} canales de entrada)',
        fontsize=13, fontweight='bold'
    )

plt.tight_layout()
plt.show()

print("\n💡 INTERPRETACIÓN:")
if num_input_channels == 3:
    print("   - Patrones claros = detectores de bordes/texturas específicas")
    print("   - Colores dominantes = sensibilidad a canales RGB")
else:
    print("   ⚠️  En capas profundas, los filtros son ABSTRACTOS")
    print("   - No verás líneas claras como en conv1")
    print("   - Operan sobre features ya procesados, no píxeles")
    print("   - Para entender qué detectan, mejor usar Feature Visualization")

print("\n" + "="*70)

In [ ]:
# ===================================================================
# CELDA: COMPARACIÓN LADO A LADO
# ===================================================================

print("="*70)
print("📊 FILTROS vs MAPAS DE ACTIVACIÓN")
print("="*70)


# Obtener filtros
layer_module = dict(model.named_modules())[layers_to_compare[0]]
filters = layer_module.weight.data.cpu()

# Obtener mapas de activación (de celda anterior)
analyzer_temp = SingleImageAnalyzer(model, layers_to_compare[0], device)
img_tensor_temp, img_vis_temp = analyzer_temp.load_image(str(image_path))
results_temp = analyzer_temp.analyze_image(img_tensor_temp)
activations_temp = results_temp['activations']
stats_temp = analyzer_temp.get_neuron_statistics(activations_temp)
top_6 = analyzer_temp.get_top_neurons(stats_temp, top_k=6)

# Visualización
fig = plt.figure(figsize=(18, 10))
gs = fig.add_gridspec(3, 6, hspace=0.4, wspace=0.3)

# Título
fig.suptitle(
    f'COMPARACIÓN: Filtros vs Mapas de Activación - {layers_to_compare[0]}',
    fontsize=14, fontweight='bold', y=0.98
)

# Fila 0: Imagen original
ax_img = fig.add_subplot(gs[0, :])
ax_img.imshow(img_vis_temp)
ax_img.set_title('Imagen Original', fontsize=12, fontweight='bold')
ax_img.axis('off')

# Fila 1: Filtros
for col_idx, neuron_idx in enumerate(top_6):
    ax = fig.add_subplot(gs[1, col_idx])
    
    filter_rgb = filters[neuron_idx]  # [3, 7, 7]
    filter_vis = filter_rgb.permute(1, 2, 0).numpy()
    filter_vis = normalize_filter(torch.from_numpy(filter_vis)).numpy()
    
    ax.imshow(filter_vis, interpolation='nearest')
    ax.set_title(f'FILTRO #{neuron_idx}\n(Patrón que busca)', fontsize=8)
    ax.axis('off')

# Fila 2: Mapas de activación
for col_idx, neuron_idx in enumerate(top_6):
    ax = fig.add_subplot(gs[2, col_idx])
    
    act_map = activations_temp[0, neuron_idx, :, :].cpu().numpy()
    
    if act_map.max() > act_map.min():
        act_map_norm = (act_map - act_map.min()) / (act_map.max() - act_map.min())
    else:
        act_map_norm = act_map
    
    ax.imshow(act_map_norm, cmap='jet', interpolation='bilinear')
    ax.set_title(f'ACTIVACIÓN #{neuron_idx}\n(Dónde lo encuentra)', fontsize=8)
    ax.axis('off')

plt.show()

analyzer_temp.cleanup()

print("\n💡 INTERPRETACIÓN:")
print("   FILTRO:     Qué patrón busca la neurona (kernel convolucional)")
print("   ACTIVACIÓN: Dónde encontró ese patrón en la imagen específica")
print("\n   ✅ En conv1 verás: bordes, colores, gradientes claros")
print("   ⚠️  En layers profundos: patrones abstractos difíciles de interpretar")

print("\n" + "="*70)

In [ ]:
# ===================================================================
# CELDA 7: Cleanup
# ===================================================================

analyzer.cleanup()
print("\n✅ Análisis completado")